# Configuration

Define settings for remote services, create `appsettings.Development.json` using `appsettings.json` as template.

You need to manually give yourself `Cognitive Services OpenAI Contributor` rights on the `Azure OpenAI` resource if you not already have it.

Also ensure to execute `azd auth login` to have access to the Azure resources.

> This example is based on [How-To: Coordinate Agent Collaboration using Agent Group Chat](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/examples/example-agent-collaboration?pivots=programming-language-csharp).


# Install and Import Required Packages
Install and import the necessary packages using NuGet.

In [ ]:
// Install the necessary packages using NuGet
#r "nuget: Azure.Identity, 1.13.1"
// Only possible with .NET 9 support
// #r "nuget: Microsoft.Extensions.Logging, 9.0.0"
// #r "nuget: Microsoft.Extensions.Logging.Console, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Binder, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.UserSecrets, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.EnvironmentVariables, 9.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.33.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.33.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.33.0"

// Import the necessary libraries
using System;
using System.Net;
using System.ComponentModel;
using System.Diagnostics;
using System.IO;
using System.Text.Json;
using System.Threading.Tasks;
using Azure.Identity;
using Microsoft.DotNet.Interactive;
// Only possible with .NET 9 support
// using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Configuration;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.Agents.History;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;

# Create Kernel Builder
Create a Kernel builder instance.

In [ ]:
var configBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("./appsettings.json", optional: false)
    .AddJsonFile("./appsettings.Development.json", optional: false);
var configuration = configBuilder.Build();

IKernelBuilder builder = Microsoft.SemanticKernel.Kernel
    .CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        configuration["ChatModelDeployment"],
        configuration["Endpoint"],
        new AzureDeveloperCliCredential());

// var loggerFactory = LoggerFactory.Create(builder =>
// {
//     builder.AddSimpleConsole(options =>
//     {
//         options.IncludeScopes = true;
//         options.SingleLine = true;
//         options.TimestampFormat = "HH:mm:ss ";
//     });
// });

// builder.Services.AddSingleton(loggerFactory);

private sealed class FunctionInvocationFilter() : IFunctionInvocationFilter
{
    public async Task OnFunctionInvocationAsync(FunctionInvocationContext context, Func<FunctionInvocationContext, Task> next)
    {
        if (context.Function.PluginName == "SearchPlugin")
        {
            Console.WriteLine($"{context.Function.Name}:{JsonSerializer.Serialize(context.Arguments)}");
        }
        await next(context);
    }
}

builder.Services.AddSingleton<IFunctionInvocationFilter, FunctionInvocationFilter>();

Microsoft.SemanticKernel.Kernel kernel = builder.Build();

public class Tools
{
    [KernelFunction]
    [Description("Copies the provided content to the console output.")]
    public static void WriteConsoleOutput(string content)
    {
        if (string.IsNullOrWhiteSpace(content))
        {
            return;
        }

        Console.Write(content);
    }
}

Microsoft.SemanticKernel.Kernel toolKernel = kernel.Clone();
toolKernel.Plugins.AddFromType<Tools>()


# Agent Definitions

In [3]:
// The Agent Framework is experimental and requires warning suppression
#pragma warning disable CA2007, IDE1006, SKEXP0001, SKEXP0110, OPENAI001

const string ReviewerName = "Reviewer";
const string WriterName = "Writer";

ChatCompletionAgent agentReviewer =
    new()
    {
        Name = ReviewerName,
        Instructions =
            """
            Your responsiblity is to review and identify how to improve user provided content.
            If the user has providing input or direction for content already provided, specify how to address this input.
            Never directly perform the correction or provide example.
            Once the content has been updated in a subsequent response, you will review the content again until satisfactory.
            Always copy satisfactory content to the console output using available tools and inform user.

            RULES:
            - Only identify suggestions that are specific and actionable.
            - Verify previous suggestions have been addressed.
            - Never repeat previous suggestions.
            """,
        Kernel = toolKernel,
        Arguments =
            new KernelArguments(
                new AzureOpenAIPromptExecutionSettings() 
                { 
                    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto() 
                })
    };

ChatCompletionAgent agentWriter =
    new()
    {
        Name = WriterName,
        Instructions =
            """
            Your sole responsiblity is to rewrite content according to review suggestions.

            - Always apply all review direction.
            - Always revise the content in its entirety without explanation.
            - Never address the user.
            """,
        Kernel = kernel,
    };

# Chat Definitions

In [4]:
// The Agent Framework is experimental and requires warning suppression
#pragma warning disable CA2007, IDE1006, SKEXP0001, SKEXP0110, OPENAI001

KernelFunction selectionFunction =
    AgentGroupChat.CreatePromptFunctionForStrategy(
        $$$"""
        Examine the provided RESPONSE and choose the next participant.
        State only the name of the chosen participant without explanation.
        Never choose the participant named in the RESPONSE.

        Choose only from these participants:
        - {{{ReviewerName}}}
        - {{{WriterName}}}

        Always follow these rules when choosing the next participant:
        - If RESPONSE is user input, it is {{{ReviewerName}}}'s turn.
        - If RESPONSE is by {{{ReviewerName}}}, it is {{{WriterName}}}'s turn.
        - If RESPONSE is by {{{WriterName}}}, it is {{{ReviewerName}}}'s turn.

        RESPONSE:
        {{$lastmessage}}
        """,
        safeParameterNames: "lastmessage");


const string TerminationToken = "yes";

KernelFunction terminationFunction =
    AgentGroupChat.CreatePromptFunctionForStrategy(
        $$$"""
        Examine the RESPONSE and determine whether the content has been deemed satisfactory.
        If content is satisfactory, respond with a single word without explanation: {{{TerminationToken}}}.
        If specific suggestions are being provided, it is not satisfactory.
        If no correction is suggested, it is satisfactory.

        RESPONSE:
        {{$lastmessage}}
        """,
        safeParameterNames: "lastmessage");        

# Creating Group Chat

In [5]:
// The Agent Framework is experimental and requires warning suppression
#pragma warning disable CA2007, IDE1006, SKEXP0001, SKEXP0110, OPENAI001

ChatHistoryTruncationReducer historyReducer = new(1);

AgentGroupChat chat =
    new(agentReviewer, agentWriter)
    {
        ExecutionSettings = new AgentGroupChatSettings
        {
            SelectionStrategy =
                new KernelFunctionSelectionStrategy(selectionFunction, kernel)
                {
                    // Always start with the editor agent.
                    InitialAgent = agentReviewer,
                    // Save tokens by only including the final response
                    HistoryReducer = historyReducer,
                    // The prompt variable name for the history argument.
                    HistoryVariableName = "lastmessage",
                    // Returns the entire result value as a string.
                    ResultParser = (result) => result.GetValue<string>() ?? agentReviewer.Name
                },
            TerminationStrategy =
                new KernelFunctionTerminationStrategy(terminationFunction, kernel)
                {
                    // Only evaluate for editor's response
                    Agents = [agentReviewer],
                    // Save tokens by only including the final response
                    HistoryReducer = historyReducer,
                    // The prompt variable name for the history argument.
                    HistoryVariableName = "lastmessage",
                    // Limit total number of turns
                    MaximumIterations = 12,
                    // Customer result parser to determine if the response is "yes"
                    ResultParser = (result) => result.GetValue<string>()?.Contains(TerminationToken, StringComparison.OrdinalIgnoreCase) ?? false
                }
        }
    };

Console.WriteLine("Ready!");

# Chat Loop for Text Review

Execute it and enter a text for review.
If no text is entered, a sample will be used.

**Commands**
- `@<file path>` - using a text file as input
- `RESET` - reset everything and start again
- `EXIT` - exit chat
- (blank) - use sampe text for review


In [ ]:
// The Agent Framework is experimental and requires warning suppression
#pragma warning disable CA2007, IDE1006, SKEXP0001, SKEXP0110, OPENAI001

var sampleTextForReview = @"Elephants are the largest land mamma4ls on earth and have distinctly massive bodies, large ears, and long trunks.
    They use their trunks to pick up objects, trumpet warnings, greet other elephants, or suck up water for drinking or bathing, among other uses.
    Both male and female African elephants grow tusks and each individual can either be left- or right-tusked, an5d the one they use more is usually
    smaller because of wear and tear. Elephant tusks serve many purposes. 
    These extended teeth can be used to protect the elephant's ztrunk, lift and move objects, gather food, and strip bark from trees. 
    They can also be used for defense. During times of droughdd, elephants even use their tusks to dig holes to find water underground.";

bool isComplete = false;
do
{
    Console.WriteLine();
    Console.Write("> ");
    var input = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Chat (insert text for review or ' '[space] for sample):");
    if (string.IsNullOrWhiteSpace(input))
    {
        Console.WriteLine("[Using sample]");
        input = sampleTextForReview;
    }
    input = input.Trim();
    if (input.Equals("EXIT", StringComparison.OrdinalIgnoreCase))
    {
        isComplete = true;
        break;
    }

    if (input.Equals("RESET", StringComparison.OrdinalIgnoreCase))
    {
        await chat.ResetAsync();
        Console.WriteLine("[Converation has been reset]");
        continue;
    }

    if (input.StartsWith("@", StringComparison.Ordinal) && input.Length > 1)
    {
        string filePath = input.Substring(1);
        try
        {
            if (!File.Exists(filePath))
            {
                Console.WriteLine($"Unable to access file: {filePath}");
                continue;
            }
            input = File.ReadAllText(filePath);
        }
        catch (Exception)
        {
            Console.WriteLine($"Unable to access file: {filePath}");
            continue;
        }
    }

    Console.WriteLine();
    Console.WriteLine($"User: {input}");
    chat.AddChatMessage(new ChatMessageContent(AuthorRole.User, input));

    chat.IsComplete = false;

    try
    {
        await foreach (ChatMessageContent response in chat.InvokeAsync())
        {
            Console.WriteLine();
            Console.WriteLine($"{response.AuthorName.ToUpperInvariant()}:{Environment.NewLine}{response.Content}");
        }
    }
    catch (HttpOperationException exception)
    {
        Console.WriteLine(exception.Message);
        if (exception.InnerException != null)
        {
            Console.WriteLine(exception.InnerException.Message);
            if (exception.InnerException.Data.Count > 0)
            {
                Console.WriteLine(JsonSerializer.Serialize(exception.InnerException.Data, new JsonSerializerOptions() { WriteIndented = true }));
            }
        }
    }
} while (!isComplete);